In [11]:
import pandas as pd
import torch
from model import PSAModel
import re
import emoji
from transformers import AutoTokenizer
from tqdm import tqdm

In [12]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

'cuda'

In [13]:
checkpoint = torch.load("anonymous/path/to/data", weights_only=False)

In [14]:
train_data = pd.read_csv("anonymous/path/to/data")
#val_data = pd.read_csv("anonymous/path/to/data")
test_data = pd.read_csv("anonymous/path/to/data")

train_data.head()

,content,labels
0,தென்காசி தொகுதி புதிய தமிழகம் கட்சி வேட்பாளர் ...,Neutral
1,அண்ணன் இதனை சூசகமாக 11 மாதங்கள் முன்பே பேட்டிய...,Substantiated
2,ஒரு வருடம் ஆகி விட்டது இந்த துயரம் நேர்ந்து......,Opinionated
3,"எடப்பாடியை கண்டுகொள்ளாத ""எடப்பாடி""🫢\n ---\nஆதர...",Positive
4,எங்களின் அரசியல் அடுத்த தலைமுறைக்குமானது \n#மக...,Opinionated


In [15]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-MLM-Sam-TLM")
model = PSAModel().to(device)
model.load_state_dict(checkpoint['model_state_dict'])

Some weights of the model checkpoint at ai4bharat/IndicBERTv2-MLM-Sam-TLM were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [16]:
final = [ [], [], [], [], [], [], [], []]

In [17]:
classes = ['Opinionated', 'Substantiated', 'Neutral', 'Negative', 'Sarcastic', 'None of the above', 'Positive']
class_to_idx = {label: i for i, label in enumerate(classes)}

In [18]:
def preprocess(text):
    text = re.sub(r"#\S+|@\S+|https?:\/\/\S+", "", text)
    text = re.sub(r"[^அ-ஹ0-9\s]", "", text)
    text = re.sub(r"[\n\r\t]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)
    return text

In [19]:
for i in tqdm(range(test_data.shape[0])):
    label = class_to_idx[test_data.loc[i, 'labels']]
    with torch.no_grad():
        inp = tokenizer(preprocess(test_data.loc[i, 'content']), max_length=512, truncation=True, padding="max_length", return_tensors="pt").to(device)
        output = model(inp['input_ids'], inp['attention_mask']).squeeze(dim=0).cpu().numpy()
        for i in range(len(output)):
            final[i].append(output[i])
        final[-1].append(label)

100%|██████████| 544/544 [00:14<00:00, 36.94it/s]


In [20]:
df = pd.DataFrame({'F0': final[0], 'F1': final[2], 'F2': final[2], 'F3': final[3], 'F4': final[4], 'F5': final[5], 'F6': final[6], 'Labels': final[7]})
df.to_csv('final_test.csv', index=False)